# Udacity Machine Learning Engineer Nanodegree
## Capstone Project: 
## Predicting the Daily Direction of the S&P500

### Completed and Submitted by Stephen Fox
### October 2016


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from time import time
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import re
from IPython.display import display

# Read data from SP500_historical.csv - data contains returns for the S&P500 and VIX index going back over 23 years
# Source: Yahoo Finance

all_data = pd.read_csv("SP500_historical.csv",header=0)
print ("Market data read successfully!")
print ("Number of data points:", len(all_data))

In [ ]:
# Print the column headers
print (all_data.dtypes.index)

In [3]:
# Convert the 'Date' column data from a string into a form that can be manipulated mathematically
# Calculate the 'Days_Since_Open' feature, which is a measure of how many days have passed since the market last opened

from datetime import datetime

all_data['Date'] = pd.to_datetime(all_data['Date'])

all_data['Days_Since_Open'] = (all_data['Date'] - all_data['Date'].shift(-1))
all_data['Days_Since_Open'] = all_data['Days_Since_Open'].astype('timedelta64[D]')

In [ ]:
# Visualize some of the key data

%matplotlib inline

import matplotlib.pyplot as plt

plot_data = all_data.SP_Close
plot_data_VIX = all_data.Vix_Close


f, axes = plt.subplots(2, 1)

axes[0].axis([5962,0,400,2200])
axes[0].plot(plot_data)
axes[0].set_ylabel('S&P500 Close')
axes[0].set_yticks([400,800,1200,1600,2000,2400])

axes[1].axis([5962,0,0,100])
axes[1].plot(plot_data_VIX)
axes[1].set_ylabel('VIX Close')
plt.xlabel('Trading Days Since Most Recent Day')

In [ ]:
plot_data_Volume = all_data.SP_Volume / 1000000

plt.axis([5962,0,14,12000])
plt.plot(plot_data_Volume)
plt.ylabel('S&P500 Volume (in millions)')
plt.xlabel('Trading Days Since Most Recent Day')

In [6]:
# Calculate the 'Break_Coming' feature, which is a measure of whether the market will close for a break on the next day

all_data['Break_Coming'] = all_data['Days_Since_Open'].shift(1)

all_data.loc[all_data['Break_Coming']<=1,'Break_Coming'] = 0
all_data.loc[all_data['Break_Coming']>1,'Break_Coming'] = 1

all_data.Break_Coming.fillna(0, inplace=True)

In [7]:
# Calculate the 'Overnight_Return' feature

all_data['Overnight_Return'] = all_data['SP_Open'] / all_data['SP_Close'].shift(-1) - 1

In [8]:
# Calculate the 'No_Overnight_Change' value. This is not a feature, but was used to identify unreliable data rows
# See accompanying report for further details

all_data['No_Overnight_Change'] = all_data['Overnight_Return']

all_data.loc[all_data['No_Overnight_Change']==0.00,'No_Overnight_Change'] = 1
all_data.loc[all_data['No_Overnight_Change']!=1,'No_Overnight_Change'] = 0

In [9]:
# Calculate the 'Overnight_VIX' feature, which measures whether the VIX has opened up or down relative to previous day

all_data['Overnight_VIX'] = all_data['Vix_Open'] / all_data['Vix_Close'].shift(-1) - 1

In [10]:
# Calculate the 'O_to_O' feature, which compares the current opening price to the previous day

all_data['O_to_O'] = all_data['SP_Open'] / all_data['SP_Open'].shift(-1) - 1

In [11]:
# Calculate 11 different trailing return features, 9 based on the S&P500 data and 2 based on VIX data

all_data['Trail_1d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-2) - 1
all_data['Trail_2d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-3) - 1
all_data['Trail_3d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-4) - 1
all_data['Trail_4d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-5) - 1
all_data['Trail_5d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-6) - 1
all_data['Trail_21d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-22) - 1
all_data['Trail_63d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-64) - 1
all_data['Trail_126d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-127) - 1
all_data['Trail_252d_Ret'] = all_data['SP_Close'].shift(-1) / all_data['SP_Close'].shift(-253) - 1

all_data['Trail_1d_VIX'] = all_data['Vix_Close'].shift(-1) / all_data['Vix_Close'].shift(-2) - 1
all_data['Trail_5d_VIX'] = all_data['Vix_Close'].shift(-1) / all_data['Vix_Close'].shift(-6) - 1

In [12]:
# Calculate 2 different trailing volume features, which measure whether the last day's volume is above or below
# recent averages

all_data['Trail_1d_Rel_Vol'] = all_data['SP_Volume'].shift(-1) / (
    all_data['SP_Volume'].rolling(window=4).mean().shift(-5)) - 1

all_data['Trail_5d_Rel_Vol'] = all_data['SP_Volume'].rolling(window=5).mean().shift(-5) / (
    all_data['SP_Volume'].rolling(window=21).mean().shift(-26)) - 1

In [13]:
# Calculate the 'Trail_1d_PtT' feature, which measure how much the S&P500 moved from high to low values on the 
# previous trading day

all_data['Trail_1d_PtT'] = (all_data['SP_High'].shift(-1) - all_data['SP_Low'].shift(-1)) / (0.5*(
        all_data['SP_Open'].shift(-1) + all_data['SP_Close'].shift(-1)))

In [14]:
# Calculate the 'Trail_1d_VIX_PtT' feature, which measure how much the VIX moved from high to low values on the 
# previous trading day

all_data['Trail_1d_VIX_PtT'] = (all_data['Vix_High'].shift(-1) - all_data['Vix_Low'].shift(-1)) / (0.5*(
        all_data['Vix_Open'].shift(-1) + all_data['Vix_Close'].shift(-1)))

In [15]:
# Calculate the 'Intraday_Increase' label, which is ultimately what the machine learner should predict, and as is a
# measure of whether the S&P500 increased on a given day (value = 1) or not (value = 0)

all_data['Intraday_Increase'] = all_data['SP_Close'] - all_data['SP_Open']

all_data.loc[all_data['Intraday_Increase']>0,'Intraday_Increase'] = 1
all_data.loc[all_data['Intraday_Increase']<=0,'Intraday_Increase'] = 0

I am now going to delete those rows that have N/A values (e.g. you cannot do a 252 day trailing return if there are fewer than 252 days trailing a given data point)

In [ ]:
noNA_data = all_data.dropna()

print ("Number of data points:", len(noNA_data))

I will also delete rows where 'No_Overnight_Change' is equal to 1. These are points where the opening value is IDENTICAL to the previous closing data, which likely reflects missing true opening prices (the chances of the market opening at the identical level as the previous close are slim to none). Most of these points came from the older portion of the data set. There were almost no points like this in the most recent 10 years:

In [ ]:
validated_data = noNA_data[noNA_data.No_Overnight_Change != 1]
print ("Number of data points:", len(validated_data))

Next, I need to drop columns that would allow the machine learning algorithm to 'know the future'. For example, it cannot be trained using SP_Close for any given day, since that would trivially allow it to calculate whether the market closed up on any given day. It can only be provided information that one would know at the start of the day (i.e. data from previous days and the opening data). This rules out High, Low and Close values for the market and the VIX and also the market volume for any given day. Finally, I will limit the analysis to relative returns, not absolute values of the indeces, given that the stock market tends to increase with time and hence comparing the absolute value of the S&P500 over a 20+ year period will not be helpful. 

In summary, the final data set will consist of only the 20 calculated features and the 1 label, for the 2,678 validated data rows.

In [ ]:
# Print the column headers of the current data set
print (validated_data.dtypes.index)

In [19]:
# Drop the columns that are not one of the 20 features or the label
data = validated_data.drop(validated_data.columns[[0,1,2,3,4,5,6,7,8,9,13]], axis=1)

In [ ]:
# Print the headers of 'data', to validate that the correct columns were dropped
print (data.dtypes.index)

In [21]:
# Optional: export the final data set that will be used for training / testing the algorithms
np.savetxt("eval-data.csv", data, delimiter=",")

In [ ]:
# Output basic statistics on the dataset
display(data.describe())

In [ ]:
# Separate features and labels:

X = pd.DataFrame(data.loc[:,'Days_Since_Open':'Trail_1d_VIX_PtT'],dtype='float32')
y = pd.DataFrame(data.loc[:,'Intraday_Increase'],dtype='float32')

# Summarize the label data
display(y.describe())

# SKLearn Supervised Learning Algorithms:

The next few blocks of code will develop and optimize three SKLearn supervised learning classification algorithms, namely Decision Trees (DT), Naive Bayes, and Support Vector Machines (SVM). This code is based on code from project 2 (student intervention) of the Udacity Machine Learning Nanodegree.

In [ ]:
# Split the data into train, validation and test sets:
# Shuffling will not be used because test data should be drawn from the latest data points, to remove any risk of
# the learning algorithms glimpsing the future

X_train = X[252:]
X_test = X[:252]

y_train = y[252:]
y_test = y[:252]

y_train = np.reshape(y_train.values,[2426,])
y_test = np.reshape(y_test.values,[252,])

describe_train = pd.DataFrame(X_train)
describe_test = pd.DataFrame(X_test)

describe_train_labels = pd.DataFrame(y_train)
describe_test_labels = pd.DataFrame(y_test)


display(describe_train.describe())
display(describe_test.describe())

display(describe_train_labels.describe())
display(describe_test_labels.describe())

In [ ]:
# Calculate the benchmark values, against which the learners will be assessed

# A suitable benchmark for the models would be comparing to the case where one simply predicts all 1s on the test set
# i.e. simply assume the market will go up every day, as it historically has on 52-54% of days in the time range
# under consideration here:

# Benchmark F1 score results from simply predicting all '1' on the test set

from sklearn.metrics import f1_score
print ("F1 score for predicting all \"up (1)\" on test set: {:.4f}".format(
    f1_score(y_test, [1]*len(y_test), pos_label=1, average='binary')))

# Benchmark accuracy score results from simply predicting all '1' on the test set

from sklearn.metrics import accuracy_score
print ("Accuracy score for predicting all \"up (1)\" on test set: {:.4f}".format(
    accuracy_score(y_test, [1]*len(y_test))))

In [26]:
# Set up a series of functions for monitoring training and testing of the various learning algorithms

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target, y_pred, pos_label=1)

def predict_labels_accuracy(clf, features, target):
    ''' Makes predictions using a fit classifier based on the accuracy score. '''
    
    y_pred = clf.predict(features)
    
    # Print and return results
    return accuracy_score(target, y_pred)   

def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("Accuracy score for training set: {:.4f}.".format(predict_labels_accuracy(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print ("Accuracy score for test set: {:.4f}.".format(predict_labels_accuracy(clf, X_test, y_test)))

In [ ]:
# Import  three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Initialize the three models
clf_A = DecisionTreeClassifier(random_state=1)
clf_B = GaussianNB()
clf_C = SVC(random_state=2)

# Set up the training set sizes
X_train_800 = X_train[:800]
y_train_800 = y_train[:800]

X_train_1600 = X_train[:1600]
y_train_1600 = y_train[:1600]

X_train_2426 = X_train[:2426]
y_train_2426 = y_train[:2426]

# Execute the 'train_predict' function for each classifier and each training set size

# loop thru models, then thru train sizes
for clf in [clf_A, clf_B, clf_C]:
    print ("\n{}: \n".format(clf.__class__.__name__))
    for n in [800, 1600, 2426]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)
        print ('-'*32)

In [ ]:
# Optimize the Decision Tree parameters by using grid search cross validation

# Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# Create the parameters list you wish to tune

parameters = {'max_depth': [1,2,3,4,5,6,7,8,9],'criterion':('gini','entropy'),'splitter':('best','random')}

# Initialize the classifier
clf = DecisionTreeClassifier(random_state=4)

# Perform grid search on the classifier using the default scoring method (accuracy)
grid_obj = GridSearchCV(clf, parameters)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a training accuracy score of {:.4f}.".format(predict_labels_accuracy(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))
print ("Tuned model has a testing accuracy score of {:.4f}.".format(predict_labels_accuracy(clf, X_test, y_test)))

print("The best parameters are %s"
      % (grid_obj.best_params_))

importances = clf.feature_importances_

feature_importance = pd.Series(importances,index=['Days_Since_Open','Break_Coming','Overnight_Return',
                                                  'Overnight_VIX','O_to_O','Trail_1d_Ret','Trail_2d_Ret',
                                                  'Trail_3d_Ret','Trail_4d_Ret','Trail_5d_Ret','Trail_21d_Ret',
                                                  'Trail_63d_Ret','Trail_126d_Ret','Trail_252d_Ret','Trail_1d_VIX',
                                                  'Trail_5d_VIX','Trail_1d_Rel_Vol','Trail_5d_Rel_Vol','Trail_1d_PtT',
                                                  'Trail_1d_VIX_PtT'])

print '\n'
print "Feature Importance:" 
print feature_importance

# Output predictions for further analysis
np.savetxt("eval-DT.csv", clf.predict(X_test), delimiter=",")

In [ ]:
# Optimize the SVM parameters by using grid search cross validation

# Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# Create the parameters list you wish to tune

parameters = {'kernel':('linear', 'rbf','sigmoid'), 'C':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
              'gamma':[.000001,.000005,.00005,.0005,.001,.005,.01,.02,.04,.05,.1,.3,.5]}

# Initialize the classifier
clf = SVC(random_state=3)

# Perform grid search on the classifier using the default scoring method (accuracy)
grid_obj = GridSearchCV(clf, parameters)

# Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a training accuracy score of {:.4f}.".format(predict_labels_accuracy(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))
print ("Tuned model has a testing accuracy score of {:.4f}.".format(predict_labels_accuracy(clf, X_test, y_test)))

print("The best parameters are %s"
      % (grid_obj.best_params_))

# Output predictions for further analysis
np.savetxt("eval-SVM.csv", clf.predict(X_test), delimiter=",")

# Neural Network Algorithm:

The next few blocks of code will develop and optimize a TensorFlow based neural network. This code is based on the assignments from the Udacity 'Deep Learning' course.

In [30]:
# Scale features (mean = 0, stdev = 1):

from sklearn import preprocessing
X_scaled = preprocessing.scale(X)

In [ ]:
# 1-hot encoding on labels (y):

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(y)
y_scaled = enc.transform(y).toarray()

y_insight = pd.DataFrame(y_scaled)

display(y_insight.describe())

y_scaled.shape

In [ ]:
# Split the data into train, validation and test sets:
# Shuffling will not be used because test data should be drawn from the latest data points, to remove any risk of
# the learning algorithms glimpsing the future

train_dataset = X_scaled[756:]
valid_dataset = X_scaled[252:756]
test_dataset = X_scaled[:252]

train_labels = y_scaled[756:]
valid_labels = y_scaled[252:756]
test_labels = y_scaled[:252]

tf.cast(train_dataset, tf.float32)
tf.cast(valid_dataset, tf.float32)
tf.cast(test_dataset, tf.float32)

tf.cast(train_labels, tf.float32)
tf.cast(valid_labels, tf.float32)
tf.cast(test_labels, tf.float32)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

describe_train = pd.DataFrame(train_dataset)
describe_valid = pd.DataFrame(valid_dataset)
describe_test = pd.DataFrame(test_dataset)

describe_train_labels = pd.DataFrame(train_labels)
describe_valid_labels = pd.DataFrame(valid_labels)
describe_test_labels = pd.DataFrame(test_labels)


display(describe_train.describe())
display(describe_valid.describe())
display(describe_test.describe())

display(describe_train_labels.describe())
display(describe_valid_labels.describe())
display(describe_test_labels.describe())

In [33]:
#Establish functions for calculating accuracy and F1 score

from sklearn.metrics import f1_score

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def F1_score(predictions, labels):
  return f1_score(np.argmax(labels, 1), np.argmax(predictions, 1))

In [34]:
# Establish TensorFlow Graph

batch_size = 1921
num_labels = 2
num_features = 20

# Add second and third hidden layer, each with fewer nodes:

num_nodes1 = 16
num_nodes2 = 8
num_nodes3 = 4

graph = tf.Graph()
with graph.as_default():

  tf.set_random_seed(8)

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset,tf.float32)
  tf_test_dataset = tf.constant(test_dataset,tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([num_features, num_nodes1],stddev=0.22))
  biases_1 = tf.Variable(tf.zeros([num_nodes1]))
  
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes1, num_nodes2],stddev=0.32))
  biases_2 = tf.Variable(tf.zeros([num_nodes2]))

  weights_3 = tf.Variable(
    tf.truncated_normal([num_nodes2, num_nodes3],stddev=0.45))
  biases_3 = tf.Variable(tf.zeros([num_nodes3]))

  weights_4 = tf.Variable(
    tf.truncated_normal([num_nodes3, num_labels],stddev=0.71))
  biases_4 = tf.Variable(tf.zeros([num_labels]))

  # Training computation - Layer 1
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  
  # Adding Dropout - Layer 1
    
  keep_prob1 = tf.placeholder(tf.float32)
  hidden1_drop = tf.nn.dropout(hidden1,keep_prob1)
    
  # Training computation - Layer 2
    
  hidden2 = tf.nn.relu(tf.matmul(hidden1_drop, weights_2) + biases_2)
  
  # Adding Dropout - Layer 2
    
  keep_prob2 = tf.placeholder(tf.float32)
  hidden2_drop = tf.nn.dropout(hidden2,keep_prob2)
    
  # Training computation - Layer 3
    
  hidden3 = tf.nn.relu(tf.matmul(hidden2_drop, weights_3) + biases_3)
  
  # Adding Dropout - Layer 3
    
  keep_prob3 = tf.placeholder(tf.float32)
  hidden3_drop = tf.nn.dropout(hidden3,keep_prob3)
    
  logits = tf.matmul(hidden3_drop, weights_4) + biases_4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +
                  tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2) +
                  tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3) +
                  tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(biases_4))
    
  # Add the regularization term to the loss.
  loss += 1e-6 * regularizers    
  
  # Optimizer - Add learning rate decay

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
        0.05, global_step, 100000, 0.98, staircase=True)
    
  optimizer = tf.train.GradientDescentOptimizer(
    learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1)+
                                                                       biases_1), weights_2)+ biases_2), weights_3)
                             + biases_3), weights_4)+ biases_4)

  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1)+
                                                                       biases_1), weights_2)+ biases_2), weights_3)
                             + biases_3), weights_4)+ biases_4)

In [ ]:
num_steps = 7001

# Set train_batches to 8 to use the full training set

train_batches = 1
train_subset = train_batches * batch_size

print('Training set restricted to the following number of samples:',train_subset)

train_smallset = train_dataset[0:train_subset,:]
train_smalllabel = train_labels[0:train_subset]

print('Training set', train_smallset.shape, train_smalllabel.shape)

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()

  print("Initialized")
    
  step_counter = []
  step_train_accuracy = []
  step_valid_accuracy = []
  step_test_accuracy = []
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
#    offset = (step * batch_size) % (train_smalllabel.shape[0] - batch_size)
    offset = 0

    # Generate a minibatch.
    batch_data = train_smallset[offset:(offset + batch_size), :]
    batch_labels = train_smalllabel[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob1: 0.95,
                 keep_prob2:0.95,keep_prob3:0.95}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))   
      print ("F1 Score: %f" % F1_score(test_prediction.eval(), test_labels))
      
      step_counter.append(step)
      step_train_accuracy.append(accuracy(predictions, batch_labels))
      step_valid_accuracy.append(accuracy(valid_prediction.eval(), valid_labels))
      step_test_accuracy.append(accuracy(test_prediction.eval(), test_labels))
    
  print '\n'
  print("FINAL Test accuracy: %.2f%%" % accuracy(test_prediction.eval(), test_labels))   
  print ("FINAL F1 Score: %f" % F1_score(test_prediction.eval(), test_labels))
    
# Output predictions for further analysis
  np.savetxt("eval-NN.csv", test_prediction.eval(), delimiter=",")

In [ ]:
plt.plot(step_train_accuracy, label = 'train')
plt.plot(step_valid_accuracy, label = 'validate')
plt.plot(step_test_accuracy, label = 'test')
plt.ylabel('Accuracy (%)')
plt.xlabel('Number of 500 Step Training Batches')
plt.legend(loc=4)

# Trading Simulation

The model's performance will now be compared to two benchmark trading strategies, over the 252 day test set (10/01/2015 - 09/29/2016):

1. Buy and hold - the index is purchased at the opening price on the first day of the test period and then sold at the closing price of the last day of the test period.

2. Buy only - since always buying has a slight (52-54%) accuracy advantage over selling, the index is bought each day at the opening price and then sold at the closing price. This strategy is repeated each day, in contrast to the 'buy and hold' approach, which involves a single buy and a single sell event.

The model itself is evaluated as follows: if the model predicts the price will close higher, then the index is bought at the open and sold at the close. If the model predicts the price will close lower, then the index is sold at the open and bought at the close.

In [ ]:
# Identify and format data needed for trading profit calculations

validated_data_limited = pd.DataFrame(validated_data.loc[:,'Date':'SP_Close'])
validated_data_limited = validated_data_limited[0:252]

# Read in the NN Model predictions:
ML_trading_data = pd.read_csv("eval-NN.csv",header=None)

# If column 1 > column 0, then the label (prediction) is '1' (i.e. S&P500 is predicted to increase that day)
ML_trading_data['Predicted_Up'] = ML_trading_data[0] + ML_trading_data[1]

ML_trading_data.loc[ML_trading_data[1]>0.5,'Predicted_Up'] = 1
ML_trading_data.loc[ML_trading_data[1]<=0.5,'Predicted_Up'] = 0

# Append the model predictions to the market data:

validated_data_limited['Predicted_Up'] = ML_trading_data['Predicted_Up']

# Reverse the dataframe order, so dates go from oldest to newest:
validated_data_limited = validated_data_limited.iloc[::-1]

# Buy and hold benchmark calculation:

validated_data_limited['Cum_Buy_Hold_Profit'] = validated_data_limited['SP_Close'] - (
    validated_data_limited['SP_Open'][251])

# Buy only benchmark calculation:

validated_data_limited['Buy_Only_Profit'] = validated_data_limited['SP_Close'] - validated_data_limited['SP_Open']
validated_data_limited['Cum_Buy_Only_Profit'] = np.cumsum(validated_data_limited['Buy_Only_Profit'])

# NN model results:
# Model profit is equal to the buy only profit, adjusted with a negative sign if the event is a sell instead:

validated_data_limited['Model_Profit'] = validated_data_limited['Buy_Only_Profit'] * (
    2*validated_data_limited['Predicted_Up'] -1)

validated_data_limited['Cum_Model_Profit'] = np.cumsum(validated_data_limited['Model_Profit'])

# Output the final cumulative profit for the ML strategy and the benchmarks:

print "Cumulative Buy and Hold Strategy Profit ($):", validated_data_limited['Cum_Buy_Hold_Profit'][0]
print "Cumulative Buy Only Strategy Profit ($):", validated_data_limited['Cum_Buy_Only_Profit'][0]
print "Cumulative Machine Learning Strategy Profit ($):", validated_data_limited['Cum_Model_Profit'][0]

In [ ]:
# Plot the cumulative performance of the ML trading strategy compared to the benchmarks:

trading_data = validated_data_limited

plt.axis([252,0,-100,1800])

plt.plot(trading_data['Cum_Buy_Hold_Profit'], label = 'buy & hold')
plt.plot(trading_data['Cum_Buy_Only_Profit'], label = 'buy only')
plt.plot(trading_data['Cum_Model_Profit'], label = 'ML trading')

plt.ylabel('Cumulative Profit ($)')
plt.xlabel('Look back Days')
plt.legend(loc=2)

In [ ]:
print "Buy Only Daily Profit Summary Statistics:"
display(validated_data_limited['Buy_Only_Profit'].describe())
print '\n'
print "ML Trading Daily Profit Summary Statistics:"
display(validated_data_limited['Model_Profit'].describe())